In [1]:
import numpy as np

# 데이터 불러오기
data = np.loadtxt("cube_prediction.xyzc")
points = data[:, :3]
labels = data[:, 3]

# 라벨이 0인 점군만 추출
target_label = 0
plane_points = points[labels == target_label]

# 개수가 너무 많으면 랜덤 샘플링
if len(plane_points) > 1000:
    idx = np.random.choice(len(plane_points), 1000, replace=False)
    sampled_points = plane_points[idx]
else:
    # 부족하면 0-padding
    sampled_points = np.zeros((1000, 3))
    sampled_points[:len(plane_points)] = plane_points


In [2]:
import json

with open("mesh_cube_fitting_result.json", "r") as f:
    result = json.load(f)

params = result["params"]
# Cylinder: direction(3), center(3), radius(1)
gt_param = np.array(params[0] + params[1] + [params[2]])  # shape: (7,)


In [3]:
import torch

def prepare_input(points, init_param, n_points=1000):
    if points.shape[0] > n_points:
        indices = np.random.choice(points.shape[0], n_points, replace=False)
        points = points[indices]
    else:
        # 패딩
        pad = np.zeros((n_points - points.shape[0], 3))
        points = np.vstack([points, pad])
    
    flat_points = points.flatten()
    input_vector = np.concatenate([flat_points, init_param])
    return torch.tensor(input_vector, dtype=torch.float32), torch.tensor(gt_param, dtype=torch.float32)


In [6]:
import torch.nn as nn

class CylinderRegressor(nn.Module):
    def __init__(self, input_dim=3*1000 + 7, output_dim=7):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
# 학습을 위한 데이터 준비
input_tensor, target_tensor = prepare_input(sampled_points, gt_param)

# 모델 및 학습 설정
model = CylinderRegressor()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# 학습 루프
n_epochs = 5000
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    pred = model(input_tensor.unsqueeze(0))  # (1, input_dim)
    loss = criterion(pred.squeeze(), target_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 500 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.6f}")


NameError: name 'input_vector' is not defined